In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from wordcloud import WordCloud

from sklearn.metrics import accuracy_score
from sklearn.datasets import fetch_20newsgroups



In [ ]:
categories = [
            'alt.atheism',
            'soc.religion.christian',
            'comp.graphics',
            'sci.med'
            ]

categories = [
             'sci.crypt',
             'sci.electronics',
             'sci.med',
             'sci.space'
            ]



In [ ]:
train_data = fetch_20newsgroups(
        subset='train',
        categories=categories,
        shuffle=True,
        random_state=42
        )

test_data = fetch_20newsgroups(
        subset='test',
        categories=categories,
        shuffle=True,
        random_state=42
        )

train_df = pd.DataFrame(
        {'Text':train_data.data, 'Category':train_data.target})
test_df = pd.DataFrame(
        {'Text':train_data.data, 'Category':train_data.target})



In [ ]:
class_names = train_df['Category'].unique()
print(class_names)

In [ ]:
print(f"Train Length: {len(train_df)}")

In [ ]:
print(f"Test Length: {len(test_df)}")

In [ ]:
full_df = train_df.append(test_df)
print(f"Full dataset length: {len(full_df)}")

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(train_df['Text'])
X_train_counts.shape

In [ ]:
count_vect.vocabulary_.get(u'20')

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)


In [ ]:
from sklearn.naive_bayes import MultinomialNB
mnb_clf = MultinomialNB().fit(X_train_tfidf, train_df['Category'])

In [ ]:
# docs_new = ['the stocks are going up', 'mobile phones are growing fast']
docs_new = test_df['Text'].to_list()
X_new_counts = count_vect.transform(docs_new)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)

mnb_predicted = mnb_clf.predict(X_new_tfidf)

In [ ]:
# print(predicted[:5])
# print(test_df['Category'].to_list()[:5])

In [ ]:
from sklearn.metrics import classification_report
print(accuracy_score(mnb_predicted, test_df['Category'].to_list()))

In [ ]:
from sklearn import svm

SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(X_train_tfidf,train_df['Category'])
svm_predicted = SVM.predict(X_new_tfidf)
print(accuracy_score(test_df['Category'].to_list(), svm_predicted))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100,warm_start=True, oob_score=True).fit(X_train_tfidf, train_df['Category'])
predicted = clf.predict(X_new_tfidf)
print(classification_report(test_df['Category'].to_list(), predicted))

In [ ]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(random_state=1, max_iter=300).fit(X_train_tfidf, train_df['Category'])
predicted = clf.predict(X_new_tfidf)
print(accuracy_score(test_df['Category'].to_list(), predicted))